In [1]:
# https://k3no.medium.com/how-to-query-in-graphql-6ebb3f7085dc
# https://github.com/sushiswap/sushiswap-subgraph

In [3]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import dataframe_image as dfi
import os

fantom = 'https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics'
# https://thegraph.com/explorer/subgraph/eerieeight/spooky-swap-exchange

## Fantom

In [5]:
sample_transport=RequestsHTTPTransport(
    url=fantom,
    verify=True,
    retries=3,
)

client = Client(transport=sample_transport)

query = gql('''
query {
  spiritswapFactories{
    pairCount
  }
}
''')

response = client.execute(query)
number_of_pairs = int(response['spiritswapFactories'][0]['pairCount'])
print(number_of_pairs)

620


In [7]:
token0list = []
token1list = []
id_list = []

for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      token0{symbol}
                      token1{symbol}
                      id
                  }
                }
                '''
        response = client.execute(gql(querystring))
        for row in response['pairs']:
            token0list.append(row['token0']['symbol'])
            token1list.append(row['token1']['symbol'])
            id_list.append(row['id'])    

df_spirit = pd.DataFrame.from_dict(list(zip(token0list,token1list,id_list)))
df_spirit.columns = ['Token 0', 'Token 1', 'id']
df_spirit['Pair'] = df_spirit['Token 0'] + '-' + df_spirit['Token 1']
df_spirit = df_spirit[['Pair', 'id', 'Token 0', 'Token 1']]
df_spirit

,Pair,id,Token 0,Token 1
0,USDC-BUSD,0x0070b5c2329103a5c278a0f3e8c248f1b706b4a8,USDC,BUSD
1,WFTM-SafeFantom,0x0134fae8f2bc78a901eadb7a0366ac03a2144fd0,WFTM,SafeFantom
2,WFTM-PILE,0x027ce309cad438ba200c1972f76afd08b458dcd5,WFTM,PILE
3,WFTM-KINGS,0x030df16a8d38444d13e633da564e62ea71899845,WFTM,KINGS
4,WFTM-GOV,0x03b36c0790a84c6382c4afda7584107c6522f97e,WFTM,GOV
...,...,...,...,...
343,WFTM-Saaa,0xfa7558260ecdc0c0473e836db600b667e94ecc7d,WFTM,Saaa
344,ALICE-WFTM,0xfcef3df7edf4789d35b63554d24b64faafa14ebe,ALICE,WFTM
345,USDC-VW,0xfe7f8c6a0c0feca36d644a4e100728463a276534,USDC,VW
346,WFTM-IODN,0xff9f2d6ac0e4e3367064cc376754e302d8ac2f82,WFTM,IODN


In [8]:
cream_borrowing_tokens = ['USDC', 'WFTM', 'DAI', 'ETH', 'BTC', 'LINK', 'SUSHI',\
'YFI',
'SNX',
'BAND',
'AAVE',
'COVER',
'CREAM',
'HEGIC']

interesting_tokens = ['WFTM', 'USDC', 'DAI', 'FUSD', 'ETH', 'fUSDT', 'FETH', 'WBTC']
combined_tokens = cream_borrowing_tokens + interesting_tokens
combined_tokens

['USDC',
 'WFTM',
 'DAI',
 'ETH',
 'BTC',
 'LINK',
 'SUSHI',
 'YFI',
 'SNX',
 'BAND',
 'AAVE',
 'COVER',
 'CREAM',
 'HEGIC',
 'WFTM',
 'USDC',
 'DAI',
 'FUSD',
 'ETH',
 'fUSDT',
 'FETH',
 'WBTC']

In [9]:
criteria = df_spirit['Token 0'].isin(combined_tokens) & \
           df_spirit['Token 1'].isin(combined_tokens)

df_spirit_filtered = df_spirit.loc[criteria,:]
df_spirit_filtered.columns = ['Pair', 'id', 'Token 0', 'Token 1']
df_spirit_filtered

,Pair,id,Token 0,Token 1
5,WFTM-CREAM,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,WFTM,CREAM
56,WFTM-BTC,0x279b2c897737a50405ed2091694f225d83f2d3ba,WFTM,BTC
71,WFTM-SNX,0x33068ad825fbeedb0e57e0466e379290b699f57c,WFTM,SNX
91,USDC-ETH,0x4556f6afa9b016a83052d459290573a5d83c8764,USDC,ETH
100,WFTM-YFI,0x4fc38a2735c7da1d71ccabf6dec235a7da4ec52c,WFTM,YFI
108,WFTM-COVER,0x5427e7ab61145c41b1e58453121245daef3929c3,WFTM,COVER
128,WFTM-ETH,0x613bf4e46b4817015c01c6bb31c7ae9edaadc26e,WFTM,ETH
137,USDC-FUSD,0x679449a920087828776aeef4074549410d5c8065,USDC,FUSD
145,USDC-BTC,0x6d134d63417dc67f57dbc9fcaad4b4cbfab3af2f,USDC,BTC
149,WFTM-ETH,0x6d87359b1e3fd852bd36250018e3328d1afd78e7,WFTM,ETH


In [10]:
df4 = pd.DataFrame([])
lenid = len(df_spirit_filtered['id'])

client = Client(transport=sample_transport)

for count, id in enumerate(df_spirit_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 7) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client.execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)
    
df4

 1 / 19 pair: 0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a
 2 / 19 pair: 0x279b2c897737a50405ed2091694f225d83f2d3ba
 3 / 19 pair: 0x33068ad825fbeedb0e57e0466e379290b699f57c
 4 / 19 pair: 0x4556f6afa9b016a83052d459290573a5d83c8764
 5 / 19 pair: 0x4fc38a2735c7da1d71ccabf6dec235a7da4ec52c
 6 / 19 pair: 0x5427e7ab61145c41b1e58453121245daef3929c3
 7 / 19 pair: 0x613bf4e46b4817015c01c6bb31c7ae9edaadc26e
 8 / 19 pair: 0x679449a920087828776aeef4074549410d5c8065
 9 / 19 pair: 0x6d134d63417dc67f57dbc9fcaad4b4cbfab3af2f
10 / 19 pair: 0x6d87359b1e3fd852bd36250018e3328d1afd78e7
11 / 19 pair: 0x9606d683d03f012dda296ef0ae9261207c4a5847
12 / 19 pair: 0x98c8cc0e444ce7490926a6c58484b937cf60e117
13 / 19 pair: 0x9fe4c0ce5f533e96c2b72d852f190961ad5a7bb3
14 / 19 pair: 0xbaf1b2fd16f7294ca158b3f1065e5f27f9c72b61
15 / 19 pair: 0xd061c6586670792331e14a80f3b3bb267189c681
16 / 19 pair: 0xd14dd3c56d9bc306322d4cea0e1c49e9ddf045d4
17 / 19 pair: 0xd3a0c65e5cefcee01fe2b10d04e7c16ea36974de
18 / 19 pair: 0xdbc490b47508d31

,dailyVolumeUSD,date,reserveUSD,id
0,28210.7360787049979815680294217629,1624060800,635394.2683036059688112701531773593,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a
1,28234.52255019068855675994163701818,1623974400,602948.9834179958054908427556955698,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a
2,45731.32970840973351778782213441872,1623888000,656317.811908282891644527329123685,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a
3,13191.55273750856377741335732310653,1623801600,637236.6478124410544370373014081818,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a
4,22530.38933803720187802103826302056,1623715200,713754.7732246866747651528403425254,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a
...,...,...,...,...
2,1057237.32792032237233483895054982,1623888000,2221636.954210931399114326060566159,0xe7e90f5a767406eff87fdad7eb07ef407922ec1d
3,603765.876257862717981704745722631,1623801600,2069496.6894562425845535934344946,0xe7e90f5a767406eff87fdad7eb07ef407922ec1d
4,596014.2240938643283064463633435821,1623715200,2147812.79341156754345674350111424,0xe7e90f5a767406eff87fdad7eb07ef407922ec1d
5,706128.2949151509962194467541889058,1623628800,2137666.226853373752370636615373126,0xe7e90f5a767406eff87fdad7eb07ef407922ec1d


In [12]:
df5 = df4.merge(df_spirit_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['dailyVolumeUSD','reserveUSD']] = df5[['dailyVolumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s')
df5['fee'] = round(df5['dailyVolumeUSD'] * 0.003,6)
df5['1y APR for 100 invested'] = round((100/df5['reserveUSD']) * df5['fee'] * 365,3)
df5['Pair'] = '[Spirit FTM] ' + df5['Pair']
df5.style.format({'reserveUSD': "{:0<4,.10f}"})

,dailyVolumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,1y APR for 100 invested
0,28210.736079,2021-06-19 00:00:00,"635,394.2683036060",0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,[Spirit FTM] WFTM-CREAM,WFTM,CREAM,84.632208,4.862000
1,28234.522550,2021-06-18 00:00:00,"602,948.9834179959",0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,[Spirit FTM] WFTM-CREAM,WFTM,CREAM,84.703568,5.128000
2,45731.329708,2021-06-17 00:00:00,"656,317.8119082829",0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,[Spirit FTM] WFTM-CREAM,WFTM,CREAM,137.193989,7.630000
3,13191.552738,2021-06-16 00:00:00,"637,236.6478124411",0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,[Spirit FTM] WFTM-CREAM,WFTM,CREAM,39.574658,2.267000
4,22530.389338,2021-06-15 00:00:00,"713,754.7732246866",0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,[Spirit FTM] WFTM-CREAM,WFTM,CREAM,67.591168,3.456000
5,31662.812234,2021-06-14 00:00:00,"724,962.1797631354",0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,[Spirit FTM] WFTM-CREAM,WFTM,CREAM,94.988437,4.782000
6,21466.656609,2021-06-13 00:00:00,"724,183.9673958843",0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,[Spirit FTM] WFTM-CREAM,WFTM,CREAM,64.399970,3.246000
7,93894.149876,2021-06-19 00:00:00,"2,139,037.9466912746",0x279b2c897737a50405ed2091694f225d83f2d3ba,[Spirit FTM] WFTM-BTC,WFTM,BTC,281.682450,4.807000
8,270674.761433,2021-06-18 00:00:00,"2,003,016.1769693517",0x279b2c897737a50405ed2091694f225d83f2d3ba,[Spirit FTM] WFTM-BTC,WFTM,BTC,812.024284,14.797000
9,313553.010926,2021-06-17 00:00:00,"2,083,730.8181634152",0x279b2c897737a50405ed2091694f225d83f2d3ba,[Spirit FTM] WFTM-BTC,WFTM,BTC,940.659033,16.477000


In [13]:
criteria1 = df5['reserveUSD'] > 1
criteria2 = df5['date'] == df5['date'].max()

latest_results = df5.loc[(criteria1 & criteria2), :].sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'])\
                    .sort_values(by = '1y APR for 100 invested',
                                 ascending = False)\
                    .reset_index(drop = True)\
                    .drop(columns = ['id', 'Token 0', 'Token 1'])\
                    .reset_index(drop = True)[['date', 'Pair', '1y APR for 100 invested',  'reserveUSD', 'dailyVolumeUSD', 'fee']]

df_styled  = latest_results.style.format({'reserveUSD': "{:0<4,.2f}",
                             'dailyVolumeUSD': "{:0<4,.2f}",
                             'fee': "{:0<4,.2f}",
                             '1y APR for 100 invested': "{:0<4,.2f}"})\
              .set_properties(subset=["Pair"], **{'text-align': 'left'})

df_styled

,date,Pair,1y APR for 100 invested,reserveUSD,dailyVolumeUSD,fee
0,2021-06-19 00:00:00,[Spirit FTM] WFTM-DAI,256.56,258.76,606.27,1.82
1,2021-06-19 00:00:00,[Spirit FTM] USDC-WFTM,35.76,"2,505,465.33","818,154.01","2,454.46"
2,2021-06-19 00:00:00,[Spirit FTM] DAI-FUSD,33.78,"1,582.08",488.10,1.46
3,2021-06-19 00:00:00,[Spirit FTM] fUSDT-WFTM,22.40,"959,094.90","196,203.71",588.61
4,2021-06-19 00:00:00,[Spirit FTM] USDC-FUSD,10.97,"41,273.01","4,134.99",12.40
5,2021-06-19 00:00:00,[Spirit FTM] WFTM-FUSD,8.69,"83,957.13","6,661.47",19.98
6,2021-06-19 00:00:00,[Spirit FTM] WFTM-YFI,8.60,"1,253,652.94","98,403.20",295.21
7,2021-06-19 00:00:00,[Spirit FTM] WFTM-ETH,8.24,"4,625,034.69","347,922.73","1,043.77"
8,2021-06-19 00:00:00,[Spirit FTM] WFTM-CREAM,4.86,"635,394.27","28,210.74",84.63
9,2021-06-19 00:00:00,[Spirit FTM] WFTM-BTC,4.81,"2,139,037.95","93,894.15",281.68


In [33]:
dfi.export(df_styled, 'df_styled_spooky_fantom.png')
os.startfile('df_styled_spooky_fantom.png')

In [14]:

# ['WFTM', 'USDC', 'DAI', 'FUSD', 'ETH', 'fUSDT', 'FETH']


# hyperswap fantom   https://ftm-info.hyperjump.fi/pairs
# honeyswap polygon  https://info.honeyswap.org/#/pairs